## Installing prerequisites

In [ ]:
!pip install patool
!pip install DadmaTools
!pip install hazm

In [2]:
import patoolib
import os
import pandas as pd
import dadmatools
from dadmatools.normalizer import Normalizer
from hazm import Lemmatizer
from hazm import Normalizer as NOrmalizer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

In [3]:
!wget https://drive.usercontent.google.com/download?id=15J4zPN1BD7Q_ZIQ39VeFquwSoW8qTxgu&export=download&authuser=0

--2024-09-17 17:55:56--  https://drive.usercontent.google.com/download?id=15J4zPN1BD7Q_ZIQ39VeFquwSoW8qTxgu
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3261257 (3.1M) [application/octet-stream]
Saving to: ‘download?id=15J4zPN1BD7Q_ZIQ39VeFquwSoW8qTxgu’

download?id=15J4zPN 100%[===================>]   3.11M  --.-KB/s    in 0.02s   

2024-09-17 17:56:00 (191 MB/s) - ‘download?id=15J4zPN1BD7Q_ZIQ39VeFquwSoW8qTxgu’ saved [3261257/3261257]



In [4]:
os.rename('download?id=15J4zPN1BD7Q_ZIQ39VeFquwSoW8qTxgu', 'snappfood.zip')

In [5]:
patoolib.extract_archive("/content/snappfood.zip",outdir="/content")

INFO patool: Extracting /content/snappfood.zip ...
INFO:patool:Extracting /content/snappfood.zip ...
INFO patool: running /usr/bin/7z x -o/content -- /content/snappfood.zip
INFO:patool:running /usr/bin/7z x -o/content -- /content/snappfood.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/snappfood.zip extracted to `/content'.
INFO:patool:... /content/snappfood.zip extracted to `/content'.


'/content'

In [ ]:
!wget https://drive.usercontent.google.com/download?id=15J4zPN1BD7Q_ZIQ39VeFquwSoW8qTxgu&export=download&authuser=0
os.rename('download?id=15J4zPN1BD7Q_ZIQ39VeFquwSoW8qTxgu', 'snappfood.zip')
patoolib.extract_archive("/content/snappfood.zip",outdir="/content")

## Preprocessing

In [7]:
df = pd.read_csv('/content/snappfood/train.csv', delimiter='\t')
df.head()

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [8]:
cols_to_remove = ['Unnamed: 0', 'label']
df.drop(cols_to_remove, axis=1, inplace=True)
df.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [9]:
df.shape

(56700, 2)

In [10]:
df['label_id'].value_counts()

,count
label_id,
1,28350
0,28350


In [11]:
normalizer1 = Normalizer(
    full_cleaning=False,
    unify_chars=False,
    refine_punc_spacing=False,
    remove_extra_space=False,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=True,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with='',
    replace_home_number_with=None
)

In [12]:
normalizer2 = NOrmalizer()

In [13]:
lemmatizer = Lemmatizer()

In [14]:
def preprocess(text):
  # normalizing with dadmatools
  text = normalizer1.normalize(text)

  # normalizing with hazm
  text = normalizer2.correct_spacing(text)
  text = normalizer2.remove_diacritics(text)
  text = normalizer2.remove_specials_chars(text)
  text = normalizer2.decrease_repeated_chars(text)
  text = normalizer2.unicodes_replacement(text)
  text = lemmatizer.lemmatize(text)

  # url remove
  text = re.sub(r'http\S +|https\S +|www\S', '', text)




  return text

In [15]:
%%time
preprocess(df['comment'][100])

CPU times: user 8.49 ms, sys: 47 µs, total: 8.54 ms
Wall time: 10.9 ms


'نان خمیر بود!!!!!'

In [16]:
df['comment'][10]

'اکلر فوق العاده بود اما بافت چیزکیک مونده بود و دوست نداشتم.'

In [17]:
df['preprocessed_comment'] = df['comment'].apply(preprocess)

In [18]:
df.head()

,comment,label_id,preprocessed_comment
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,حیف وقت بنویسم سرویس دهیتون افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0,ساعته برسه نیم ساعت موقع رسید، ببین چقدر پلاک ...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1,قیمت مدل کیفیتش سازگاری نداره، ظاهر فریبنده دا...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0,عالی اندازه کیفیت خوب، امیداورم کیفیتتون باشه ...
4,شیرینی وانیلی فقط یک مدل بود.,0,شیرینی وانیلی مدل بود.


## Train

In [19]:
X = df['preprocessed_comment']
y = df['label_id']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
CV = CountVectorizer()
X_train_vec = CV.fit_transform(X_train)
X_test_vec = CV.transform(X_test)

In [21]:
model_LR=LogisticRegression()
model_LR.fit(X_train_vec,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
y_pred_LR = model_LR.predict(X_test_vec)
accuracy_score(y_test, y_pred_LR)

0.8117283950617284

In [23]:
print(classification_report(y_test, y_pred_LR))

              precision    recall  f1-score   support

           0       0.82      0.79      0.81      5613
           1       0.80      0.83      0.82      5727

    accuracy                           0.81     11340
   macro avg       0.81      0.81      0.81     11340
weighted avg       0.81      0.81      0.81     11340



In [24]:
print(confusion_matrix(y_test, y_pred_LR))

[[4441 1172]
 [ 963 4764]]
